In [1]:
addprocs(10)
using AdaGram

  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubuntu/.julia/v0.5/AdaGram/src/AdaGram.jl:49
  likely near /home/ubunt

In [33]:
train_fn  =  "../../eval/data/corpora/WikiCorp/tokenised_lowercase_WestburyLab.wikicorp.201004.txt" #"training text data"
output_fn = "../../eval/models/adagram/v1_d100.adagram_model"#"file to save the model (in Julia format)"
dict_fn = "../../eval/data/corpora/WikiCorp/tokenised_lowercase_WestburyLab.wikicorp.201004.1gram" #"dictionary file with word frequencies"

window = 4 #"(max) window size"
min_freq  = 20 #"min. frequency of the word"
remove_top_k = 0 #"remove top K most frequent words"
dim  = 100 #"dimensionality of representations"
prototypes = 5 #"number of word prototypes"
alpha = 0.1 #"prior probability of allocating a new prototype"
d  = 0.0 #"parameter of Pitman-Yor process"
subsample = 1e-5 #"subsampling treshold. useful value is 1e-5"
context_cut  = true #"randomly reduce size of the context"
epochs = 1 #"number of epochs to train"
initcount = 1. #"initial weight (count) on first sense for each word"
stopwords =Set{AbstractString}() #"list of stop words"
sense_treshold = 1e-10 #"minimal probability of a meaning to contribute into gradients"
save_treshold = 1e-3 #"minimal probability of a meaning to save after training"
regex = r"" #"ignore words not matching provided regex"


r""

In [3]:
print("Building dictionary... ")
vm, dict = read_from_file(dict_fn, dim, prototypes, min_freq,remove_top_k, stopwords; regex=regex)
println("Done!")


Building dictionary... Done!


In [4]:
vm.alpha = alpha
vm.d = d

0.0

In [ ]:
inplace_train_vectors!(vm, dict, train_fn, window;
                       threshold=subsample, context_cut=context_cut,
                       epochs=epochs, init_count=initcount, sense_treshold=sense_treshold)



 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in remotecall(::Int64, ::Function, ::Int64, ::Vararg{Int64,N}) at ./deprecated.jl:50
 in #inplace_train_vectors!#18(::Int64, ::Float64, ::Void, ::Float64, ::Bool, ::Int64, ::Float64, ::Float64, ::AdaGram.#inplace_train_vectors!, ::AdaGram.VectorModel, ::AdaGram.Dictionary, ::String, ::Int64) at /home/ubuntu/.julia/v0.5/AdaGram/src/gradient.jl:154
 in (::AdaGram.#kw##inplace_train_vectors!)(::Array{Any,1}, ::AdaGram.#inplace_train_vectors!, ::AdaGram.VectorModel, ::AdaGram.Dictionary, ::String, ::Int64) at ./<missing>:0
 in include_string(::String, ::String) at ./loading.jl:380
 in execute_request_0x535c5df2(::ZMQ.Socket, ::IJulia.Msg) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:183
 in eventloop(::ZMQ.Socket) at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:143
 in (::IJulia.##26#32)() at ./task.jl:309
while loading In[5], in expression starting on line 1


	From worker 2:	64000 words read, 1296374/596129680
	From worker 2:	0.02% -11.1842 0.0250 0.0250 2.14/3.00 3.13 kwords/sec
	From worker 2:	0.02% -11.1178 0.0250 0.0250 2.00/3.00 3.84 kwords/sec
	From worker 2:	0.02% -11.1022 0.0250 0.0250 1.91/3.00 3.90 kwords/sec
	From worker 2:	0.02% -11.0636 0.0250 0.0250 1.84/3.00 3.92 kwords/sec
	From worker 3:	64000 words read, 597414099/1192259360
	From worker 2:	0.03% -11.0132 0.0250 0.0250 1.78/3.00 3.91 kwords/sec


In [34]:

save_model(output_fn, vm, dict, save_treshold)

In [1]:
word = "sweet"
prior_probs = expected_pi(vm, dict.word2id[word])
for ii in 1:5
    println(prior_probs[ii],"\t",nearest_neighbors(vm, dict, word, ii, 10))
    println()
end

LoadError: LoadError: UndefVarError: expected_pi not defined
while loading In[1], in expression starting on line 2

In [59]:
?names

search: names fieldnames dirname tempname fullname basename TypeName fieldname



```
names(x::Module[, all=false[, imported=false]])
```

Get an array of the names exported by a `Module`, with optionally more `Module` globals according to the additional parameters.


In [1]:
using Utils


In [2]:
blk = quote
    train_fn  =  "../../eval/data/corpora/WikiCorp/tokenised_lowercase_WestburyLab.wikicorp.201004.txt" #"training text data"
output_fn = "../../eval/models/adagram/v1_d100.adagram_model"#"file to save the model (in Julia format)"
dict_fn = "../../eval/data/corpora/WikiCorp/tokenised_lowercase_WestburyLab.wikicorp.201004.1gram" #"dictionary file with word frequencies"

window = 4 #"(max) window size"
min_freq  = 20 #"min. frequency of the word"
remove_top_k = 0 #"remove top K most frequent words"
dim  = 100 #"dimensionality of representations"
prototypes = 5 #"number of word prototypes"
alpha = 0.1 #"prior probability of allocating a new prototype"
d  = 0.0 #"parameter of Pitman-Yor process"
subsample = 1e-5 #"subsampling treshold. useful value is 1e-5"
context_cut  = true #"randomly reduce size of the context"
epochs = 1 #"number of epochs to train"
initcount = 1. #"initial weight (count) on first sense for each word"
stopwords =Set{AbstractString}() #"list of stop words"
sense_treshold = 1e-10 #"minimal probability of a meaning to contribute into gradients"
save_treshold = 1e-3 #"minimal probability of a meaning to save after training"
#regex = r"" #"ignore words not matching provided regex"
    end |> dump

Expr
  head: Symbol block
  args: Array{Any}((34,))
    1: Expr
      head: Symbol line
      args: Array{Any}((2,))
        1: Int64 2
        2: Symbol In[2]
      typ: Any
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol train_fn
        2: String "../../eval/data/corpora/WikiCorp/tokenised_lowercase_WestburyLab.wikicorp.201004.txt"
      typ: Any
    3: Expr
      head: Symbol line
      args: Array{Any}((2,))
        1: Int64 3
        2: Symbol In[2]
      typ: Any
    4: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol output_fn
        2: String "../../eval/models/adagram/v1_d100.adagram_model"
      typ: Any
    5: Expr
      head: Symbol line
      args: Array{Any}((2,))
        1: Int64 4
        2: Symbol In[2]
      typ: Any
    ...
    30: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol stopwords
        2: Expr
          head: Symbol call
          args: Array{Any}((1,))
            1: Expr
 

In [44]:
reload("Utils")

In [47]:
base_name = "paper_v1btempexample"
param_save_fn =  "../models/adagram/$(base_name).params.jld"
output_fn = "../models/adagram/$(base_name).adagram_model"#"file to save the model (in Julia format)"
@assert !isfile(output_fn)
@param_save param_save_fn begin
        nprocessors = nprocs()
        train_fn  =  "../data/corpora/WikiCorp/tokenised_lowercase_WestburyLab.wikicorp.201004.txt" #"training text data"
        output_fn = output_fn #file to save the model (in Julia format)"
        dict_fn = "../data/corpora/WikiCorp/tokenised_lowercase_WestburyLab.wikicorp.201004.1gram" #"dictionary file with word frequencies"

        window = 10 #"(max) window size" C in the paper
        min_freq  = 20 #"min. frequency of the word"
        remove_top_k = 0 #"remove top K most frequent words"
        dim  = 300 #"dimensionality of representations"
        prototypes = 5 #"number of word prototypes" T in the paper
        alpha = 0.15 #"prior probability of allocating a new prototype"
        d  = 0.0 #"parameter of Pitman-Yor process" D in paper
        subsample = 1e-5 #"subsampling treshold. useful value is 1e-5"
        context_cut  = true #"randomly reduce size of the context"
        epochs = 1 #"number of epochs to train"
        initcount = 1. #"initial weight (count) on first sense for each word"
        stopwords = Set{AbstractString}() #"list of stop words"
        sense_treshold = 1e-10 #"minimal probability of a meaning to contribute into gradients"
        save_treshold = 0.0 #"minimal probability of a meaning to save after training"
end


Paramaters -- saved to ../models/adagram/paper_v1btempexample.params.jld
----------
train_fn = ../data/corpora/WikiCorp/tokenised_lowercase_WestburyLab.wikicorp.201004.txt
alpha = 0.15
subsample = 1.0e-5
sense_treshold = 1.0e-10
initcount = 1.0
nprocessors = 1
epochs = 1
window = 10
output_fn = ../models/adagram/paper_v1btempexample.adagram_model
min_freq = 20
prototypes = 5
context_cut = true
stopwords = Set{AbstractString}()
remove_top_k = 0
save_treshold = 0.0
dict_fn = ../data/corpora/WikiCorp/tokenised_lowercase_WestburyLab.wikicorp.201004.1gram
dim = 300
d = 0.0
----------


In [166]:
JLD.load("../../eval/models/adagram/v1_d100_2.params.jld")

Dict{String,Any} with 1 entry:
  "save_treshold" => 6

In [51]:
 println("Using ", nprocs(), " processors")

Using 11 processors


In [13]:
begin 
     a=2
end
a

2

In [15]:
names(current_module())

37-element Array{Symbol,1}:
 Symbol("@param_save")
 :Base                
 :Blosc               
 :Compat              
 :Core                
 :FileIO              
 :HDF5                
 :IJulia              
 :IPythonDisplay      
 :JLD                 
 :JSON                
 :LegacyStrings       
 :Main                
 :Nettle              
 :Utils               
 :ZMQ                 
 :a                   
 :add_to_path         
 :alpha               
 :blk                 
 :context_cut         
 :d                   
 :dict_fn             
 :dim                 
 :epochs              
 :initcount           
 :min_freq            
 :names_candidates    
 :output_fn           
 :prototypes          
 :remove_top_k        
 :save_treshold       
 :sense_treshold      
 :stopwords           
 :subsample           
 :train_fn            
 :wixndow             

In [16]:
current_module()

Main

In [21]:
Base.eval(:a)

LoadError: LoadError: UndefVarError: a not defined
while loading In[21], in expression starting on line 1

In [24]:
current_module().eval(:a)

2